In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import openpyxl
import math

In [2]:
# list of team names currently in associated leagues (including relegated teams from last season to fill DF), as they are displayed on Understat.

epl_rank = ['Manchester City', 'Arsenal', 'Liverpool', 'Manchester United', 'Newcastle United', 'Tottenham', 'Brighton',
            'Aston Villa', 'Brentford', 'Chelsea', 'Fulham', 'Crystal Palace', 'Wolverhampton Wanderers', 'Everton',
            'West Ham', 'Nottingham Forest', 'Bournemouth', 'Leicester', 'Southampton','Burnley','Luton','Sheffield United',
            'Leeds']
ligue1_rank = ['Ajaccio', 'Marseille', 'Auxerre', 'Lens', 'Monaco', 'Toulouse', 'Nantes', 'Angers', 'Lorient',
               'Strasbourg', 'Nice', 'Lyon', 'Paris Saint Germain', 'Clermont Foot', 'Brest', 'Rennes', 'Reims',
               'Montpellier', 'Troyes', 'Lille','Metz','Le Havre']
bundesliga_rank = ['Borussia Dortmund', 'Mainz 05', 'RasenBallsport Leipzig', 'Schalke 04', 'Union Berlin',
                   'Werder Bremen', 'FC Cologne', 'Bayern Munich', 'Borussia M.Gladbach', 'Augsburg',
                   'Eintracht Frankfurt', 'Freiburg', 'Wolfsburg', 'Hertha Berlin', 'Bochum', 'Bayer Leverkusen',
                   'VfB Stuttgart', 'Hoffenheim','FC Heidenheim','Darmstadt']
serieA_rank = ['Sassuolo', 'Fiorentina', 'Torino', 'Inter', 'Cremonese', 'Salernitana', 'Empoli', 'Lazio', 'Napoli',
               'Sampdoria', 'Atalanta', 'Monza', 'Lecce', 'Bologna', 'AC Milan', 'Verona', 'Roma', 'Spezia', 'Udinese',
               'Juventus','Cagliari','Frosinone','Genoa']
laliga_rank = ['Real Madrid', 'Athletic Club', 'Villarreal', 'Athletico Madrid', 'Celta Vigo', 'Barcelona',
               'Real Valladolid', 'Getafe', 'Elche', 'Cadiz', 'Espanyol', 'Almeria', 'Osasuna', 'Girona', 'Real Betis',
               'Valencia', 'Mallorca', 'Rayo Vallecano', 'Real Sociedad', 'Sevilla','Granada','Alaves','Las Palmas']

In [3]:
# Match IDs on Understat:

# 2022/23
# ENG - 18202 - 18581 (380, 38*10)
eng22_23 = [a + 18202 for a in range(0,380)]
eng22_23_count = 0
# FRA - 19648 - 20027 (380, 38*10)
fra22_23 = [a + 19648 for a in range(0,380)] #20 teams last season
fra22_23_count = 0
# GER - 19342 - 19647 (306, 34*9)
ger22_23 = [a + 19342 for a in range(0,306)]
ger22_23_count = 0
# ITA - 18582 - 18961 (380, 38*10)
ita22_23 = [a + 18582 for a in range(0,380)]
ita22_23_count = 0
# SPA - 18962 - 19341 (380, 38*10)
spa22_23 = [a + 18962 for a in range(0,380)]
spa22_23_count = 0
# RUS - 20028 - 20267 (240, 30*8) - data ignored for now

#2023/24
# ENG - 21925 - 22304
eng23_24 = [a + 21925 for a in range(0,380)]
eng23_24_count = 0
# FRA - 23371 - 23676
fra23_24 = [a + 23371 for a in range(0,306)] #18 teams this season
fra23_24_count = 0
# GER - 23065 - 23370
ger23_24 = [a + 23065 for a in range(0,306)]
ger23_24_count = 0
# ITA - 22305 - 22684
ita23_24 = [a + 22305 for a in range(0,380)]
ita23_24_count = 0
# SPA - 22685 - 23064
spa23_24 = [a + 22685 for a in range(0,380)]
spa23_24_count = 0
# RUS - 21685 - 21924 - data ignored for now

eng_df = pd.DataFrame()
fra_df = pd.DataFrame()
ger_df = pd.DataFrame()
ita_df = pd.DataFrame()
spa_df = pd.DataFrame()                        


In [4]:
# webscraping understat for match data

def matchstat_generator(league_count,ov_df):
    
    # reset df
    ov_df = ov_df.drop(ov_df.index, inplace=True)

    for match_id in league_count:
    
        base_url ='https://understat.com/match/'
        url = base_url+str(match_id)
        
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        scripts = soup.find_all('script')

        strings = scripts[1].string

        ind_start = strings.index("('")+2
        ind_end = strings.index("')")
        json_data = strings[ind_start:ind_end]
        json_data = json_data.encode('utf8').decode('unicode_escape')

        data = json.loads(json_data)

        date = []
        x = []
        y = []
        xG = []
        result = []
        team = []
        minute = []
        data_away = data['a']
        data_home = data['h']

        for index in range(len(data_home)):
            for key in data_home[index]:
                # print(key)
                if key == 'date':
                    date.append(data_home[index][key])
                if key == 'X':
                    x.append(data_home[index][key])
                if key == 'Y':
                    y.append(data_home[index][key])
                if key == 'h_team':
                    team.append(data_home[index][key])
                if key == 'xG':
                    xG.append(float(data_home[index][key]))
                if key == 'result':
                    result.append(data_home[index][key])
                if key == 'minute':
                    minute.append(data_home[index][key])

        for index in range(len(data_away)):
            for key in data_away[index]:
                if key == 'date':
                    date.append(data_away[index][key])
                if key == 'X':
                    x.append(data_away[index][key])
                if key == 'Y':
                    y.append(data_away[index][key])
                if key == 'a_team':
                    team.append(data_away[index][key])
                if key == 'xG':
                    xG.append(float(data_away[index][key]))
                if key == 'result':
                    result.append(data_away[index][key])
                if key == 'minute':
                    minute.append(data_away[index][key])

        col_names = ['date','x','y','xG','result','team','minute']

        df_orig = pd.DataFrame([date,x,y,xG,result,team,minute],index=col_names)

        df = df_orig.T
        df['minute'] = df['minute'].astype(int)
        

        # team retrieval
        team_list = []
        sorted_team_list = []
        team_list.append(df.team.unique())
        home_team = team_list[0][0]
        away_team = team_list[0][1]
        sorted_team_list.append(home_team)
        sorted_team_list.append(away_team)
        sorted_team_list.sort()
        date = date[0]

        # xG and xGC sum
        expected_data_sums = df.groupby([team]).xG.sum().reset_index()

        if sorted_team_list[0] == home_team:
            xG_home = round(expected_data_sums.xG[0],3)
            xG_away = round(expected_data_sums.xG[1],3)
        else:
            xG_home = round(expected_data_sums.xG[1],3)
            xG_away = round(expected_data_sums.xG[0],3)

        # Shots faced sum
        shot_data_sums = df.groupby([team]).xG.count().reset_index()

        if sorted_team_list[0] == home_team:
            shots_h = shot_data_sums.xG[0]
            shots_a = shot_data_sums.xG[1]
        else:
            shots_h = shot_data_sums.xG[1]
            shots_a = shot_data_sums.xG[0]

        # Quality shots faced sum (over .25 xG)
        h_quality_shot_data_sums = len(df[(df["team"] == home_team) & (df["xG"] >= 0.25)])
        a_quality_shot_data_sums = len(df[(df["team"] == away_team) & (df["xG"] >= 0.25)])

        #goals
        home_array = [home_team]
        away_array = [away_team]
        home_goal_loc = df.loc[(df['result'] == 'Goal') & df['team'].isin(home_array)] 
        away_goal_loc = df.loc[(df['result'] == 'Goal') & df['team'].isin(away_array)] 
        away_own_goal_loc = df.loc[(df['result'] == 'OwnGoal') & df['team'].isin(away_array)]
        home_own_goal_loc = df.loc[(df['result'] == 'OwnGoal') & df['team'].isin(home_array)]
        home_goal_loc = home_goal_loc.merge(away_own_goal_loc,how='outer')
        away_goal_loc = away_goal_loc.merge(home_own_goal_loc,how='outer')
        first_goal_h = home_goal_loc.minute.min()
        first_goal_a = away_goal_loc.minute.min()

        if math.isnan(first_goal_h) is False and math.isnan(first_goal_a) is False:
            if first_goal_h > first_goal_a:
                first_h = 0
                first_a = 1
            elif first_goal_h < first_goal_a:
                first_h = 1
                first_a = 0
            else:
                first_h = 2
                first_a = 2
        elif math.isnan(first_goal_h) is True and math.isnan(first_goal_a) is False:
            first_h = 0
            first_a = 1
        elif math.isnan(first_goal_h) is False and math.isnan(first_goal_a) is True:
            first_h = 1
            first_a = 0
        else:
            first_h = 0
            first_a = 0


        home_goals = len(home_goal_loc)
        away_goals = len(away_goal_loc)

        # points
        if home_goals > away_goals:
            points_home = 3
            points_away = 0
        elif home_goals < away_goals:
            points_home = 0
            points_away = 3
        else:
            points_home = 1
            points_away = 1
        
        # id
        game_id_h = home_team+str(match_id)
        game_id_a = away_team+str(match_id)

        # btts
        if home_goals >= 1 and away_goals >= 1:
            btts = 1
            cs = 0
            blank = 0
        elif home_goals >= 1 and away_goals == 0:
            btts = 0
            cs = 1
            blank = 0
        elif home_goals == 0 and away_goals >= 1:
            btts = 0
            cs = 0
            blank = 1
        else:
            btts = 0
            cs = 1
            blank = 1

        df2_1 = pd.DataFrame([[date,game_id_h,home_team,"h",xG_home,xG_away,home_goals,away_goals,shots_h,shots_a,h_quality_shot_data_sums,a_quality_shot_data_sums,points_home,away_team,btts,blank,cs,first_h,first_a]],columns=['date','ID','Team','H/A','xG','xGC','G','GC','S','SC','QS','QSC','Pts','Opp','BTTS','Blank','CS','first_goal','first_conc'])
        df2_2 = pd.DataFrame([[date,game_id_a,away_team,"a",xG_away,xG_home,away_goals,home_goals,shots_a,shots_h,a_quality_shot_data_sums,h_quality_shot_data_sums,points_away,home_team,btts,cs,blank,first_a,first_h]],columns=['date','ID','Team','H/A','xG','xGC','G','GC','S','SC','QS','QSC','Pts','Opp','BTTS','Blank','CS','first_goal','first_conc'])
        df3 = pd.concat([df2_1,df2_2])
        ov_df = pd.concat([ov_df,df3])

    return ov_df   

In [5]:
# Generating fluid list of last season worth of games 

eng_ov = sorted(eng22_23 + eng23_24)
eng_active = []
eng_count = 0

fra_ov = sorted(fra22_23 + fra23_24)
fra_active = []
fra_count = 0

ger_ov = sorted(ger22_23 + ger23_24)
ger_active = []
ger_count = 0

ita_ov = sorted(ita22_23 + ita23_24)
ita_active = []
ita_count = 0

spa_ov = sorted(spa22_23 + spa23_24)
spa_active = []
spa_count = 0

def id_list_update_380(ov,active,counter):
    while counter < 380:
        for item in reversed(ov):
            base_url ='https://understat.com/match/'
            url = base_url+str(item)
            res = requests.get(url)
            soup = BeautifulSoup(res.content, 'lxml')
            scripts = soup.find_all('script')
            if counter == 380:
                break
            else:
                if len(scripts) != 0:
                    active.append(item)
                    counter +=1
    return active

def id_list_update_306(ov,active,counter):
    while counter < 306:
        for item in reversed(ov):
            base_url ='https://understat.com/match/'
            url = base_url+str(item)
            res = requests.get(url)
            soup = BeautifulSoup(res.content, 'lxml')
            scripts = soup.find_all('script')
            if counter == 306:
                break
            else:
                if len(scripts) != 0:
                    active.append(item)
                    counter +=1
    return active

eng_active = id_list_update_380(eng_ov,eng_active,eng_count)
fra_active = id_list_update_380(fra_ov,fra_active,fra_count)
ger_active = id_list_update_306(ger_ov,ger_active,ger_count)
ita_active = id_list_update_380(ita_ov,ita_active,ita_count)
spa_active = id_list_update_380(spa_ov,spa_active,spa_count)      

In [10]:
eng_df = matchstat_generator(eng_active, eng_df)
# with pd.ExcelWriter("../EDA_DF/EPL_understat.xlsx") as writer: #For VSCode use locally
with pd.ExcelWriter("EDA_DF/EPL_understat.xlsx") as writer: #For GitHub Actions use externally
    eng_df.to_excel(writer, sheet_name="gamesheet")
eng_df

,date,ID,Team,H/A,xG,xGC,G,GC,S,SC,QS,QSC,Pts,Opp,BTTS,Blank,CS,first_goal,first_conc
0,2023-04-26 19:00:00,Arsenal18531,Arsenal,a,0.398,2.012,1,4,8,14,0,3,0,Manchester City,1,0,0,0,1
0,2023-04-26 19:00:00,Manchester City18531,Manchester City,h,2.012,0.398,4,1,14,8,3,0,3,Arsenal,1,0,0,1,0
0,2023-04-29 11:30:00,West Ham18536,West Ham,a,2.059,2.319,3,4,8,16,3,3,0,Crystal Palace,1,0,0,1,0
0,2023-04-29 11:30:00,Crystal Palace18536,Crystal Palace,h,2.319,2.059,4,3,16,8,3,3,3,West Ham,1,0,0,0,1
0,2023-04-29 14:00:00,Brentford18534,Brentford,h,1.375,0.965,2,1,14,5,1,2,3,Nottingham Forest,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2024-04-21 14:00:00,Crystal Palace22227,Crystal Palace,h,3.863,1.345,5,2,19,5,7,2,3,West Ham,1,0,0,1,0
0,2024-04-21 14:00:00,Aston Villa22225,Aston Villa,h,2.940,2.127,3,1,14,14,4,3,3,Bournemouth,1,0,0,0,1
0,2024-04-21 14:00:00,Bournemouth22225,Bournemouth,a,2.127,2.940,1,3,14,14,3,4,0,Aston Villa,1,0,0,1,0
0,2024-04-21 15:30:00,Fulham22229,Fulham,h,0.851,1.024,1,3,12,14,1,0,0,Liverpool,1,0,0,0,1


In [8]:
fra_df = matchstat_generator(fra_active, fra_df)
# with pd.ExcelWriter("../EDA_DF/Ligue1_understat.xlsx") as writer: # For VSCode use locally
with pd.ExcelWriter("EDA_DF/Ligue1_understat.xlsx") as writer: # For GitHub Actions use externally
    fra_df.to_excel(writer, sheet_name="gamesheet")
fra_df

,date,ID,Team,H/A,xG,xGC,G,GC,S,SC,QS,QSC,Pts,Opp,BTTS,Blank,CS,first_goal,first_conc
0,2023-03-10 20:00:00,Lyon19915,Lyon,a,1.058,2.353,3,3,6,8,2,4,1,Lille,1,0,0,0,1
0,2023-03-10 20:00:00,Lille19915,Lille,h,2.353,1.058,3,3,8,6,4,2,1,Lyon,1,0,0,1,0
0,2023-03-11 20:00:00,Paris Saint Germain19917,Paris Saint Germain,a,2.674,0.751,2,1,17,7,4,1,3,Brest,1,0,0,1,0
0,2023-03-11 20:00:00,Brest19917,Brest,h,0.751,2.674,1,2,7,17,1,4,0,Paris Saint Germain,1,0,0,0,1
0,2023-03-12 19:45:00,Strasbourg19916,Strasbourg,a,2.232,1.718,2,2,24,7,2,2,1,Marseille,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2024-04-21 15:05:00,Brest23640,Brest,h,0.641,1.313,0,2,12,8,0,1,0,Monaco,0,1,0,0,1
0,2024-04-21 17:00:00,Toulouse23633,Toulouse,h,0.655,1.095,2,2,10,11,1,1,1,Marseille,1,0,0,0,1
0,2024-04-21 17:00:00,Marseille23633,Marseille,a,1.095,0.655,2,2,11,10,1,1,1,Toulouse,1,0,0,1,0
0,2024-04-21 19:00:00,Paris Saint Germain23632,Paris Saint Germain,h,3.178,0.667,4,1,14,17,5,0,3,Lyon,1,0,0,1,0


In [9]:
ger_df = matchstat_generator(ger_active, ger_df)
# with pd.ExcelWriter("../EDA_DF/Bundesliga_understat.xlsx") as writer: # For VSCode use locally
with pd.ExcelWriter("EDA_DF/Bundesliga_understat.xlsx") as writer: #For GitHub Actions use externally
    ger_df.to_excel(writer, sheet_name="gamesheet")
ger_df

,date,ID,Team,H/A,xG,xGC,G,GC,S,SC,QS,QSC,Pts,Opp,BTTS,Blank,CS,first_goal,first_conc
0,2024-04-20 13:30:00,FC Heidenheim23334,FC Heidenheim,h,1.408,1.581,1,2,10,12,2,1,0,RasenBallsport Leipzig,1,0,0,0,1
0,2024-04-20 13:30:00,RasenBallsport Leipzig23334,RasenBallsport Leipzig,a,1.581,1.408,2,1,12,10,1,2,3,FC Heidenheim,1,0,0,1,0
0,2024-04-21 13:30:00,Werder Bremen23333,Werder Bremen,h,1.408,3.324,2,1,9,14,2,5,3,VfB Stuttgart,1,0,0,1,0
0,2024-04-21 13:30:00,VfB Stuttgart23333,VfB Stuttgart,a,3.324,1.408,1,2,14,9,5,2,0,Werder Bremen,1,0,0,0,1
0,2024-04-20 13:30:00,Hoffenheim23332,Hoffenheim,h,2.829,1.364,4,3,28,13,3,2,3,Borussia M.Gladbach,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2023-05-06 13:30:00,RasenBallsport Leipzig19614,RasenBallsport Leipzig,a,2.069,0.156,1,0,13,3,4,0,3,Freiburg,0,0,1,1,0
0,2023-05-05 18:30:00,Bayer Leverkusen19613,Bayer Leverkusen,h,0.730,1.028,1,2,13,7,0,2,0,FC Cologne,1,0,0,0,1
0,2023-05-05 18:30:00,FC Cologne19613,FC Cologne,a,1.028,0.730,2,1,7,13,2,0,3,Bayer Leverkusen,1,0,0,1,0
0,2023-05-07 15:30:00,Borussia Dortmund19612,Borussia Dortmund,h,5.137,0.946,6,0,20,10,8,1,3,Wolfsburg,0,0,1,1,0


In [11]:
ita_df = matchstat_generator(ita_active, ita_df)
# with pd.ExcelWriter("../EDA_DF/SerieA_understat.xlsx") as writer: # For VSCode use locally
with pd.ExcelWriter("EDA_DF/SerieA_understat.xlsx") as writer:  # For GitHub ACtions use externally
    ita_df.to_excel(writer, sheet_name="gamesheet")
ita_df

,date,ID,Team,H/A,xG,xGC,G,GC,S,SC,QS,QSC,Pts,Opp,BTTS,Blank,CS,first_goal,first_conc
0,2024-04-21 10:30:00,Sassuolo22634,Sassuolo,h,0.410,2.366,0,3,10,14,0,4,0,Lecce,0,1,0,0,1
0,2024-04-21 10:30:00,Lecce22634,Lecce,a,2.366,0.410,3,0,14,10,4,0,3,Sassuolo,0,0,1,1,0
0,2024-04-21 13:00:00,Torino22632,Torino,h,0.521,0.579,0,0,13,12,0,0,1,Frosinone,0,1,1,0,0
0,2024-04-21 13:00:00,Frosinone22632,Frosinone,a,0.579,0.521,0,0,12,13,0,0,1,Torino,0,1,1,0,0
0,2024-04-21 16:00:00,Salernitana22631,Salernitana,h,0.330,2.678,0,2,5,12,0,5,0,Fiorentina,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2023-05-03 16:00:00,Torino18910,Torino,a,2.160,0.254,2,0,12,7,3,0,3,Sampdoria,0,0,1,1,0
0,2023-05-03 16:00:00,Salernitana18909,Salernitana,h,0.996,1.689,3,3,4,12,1,2,1,Fiorentina,1,0,0,1,0
0,2023-05-03 16:00:00,Fiorentina18909,Fiorentina,a,1.689,0.996,3,3,12,4,2,1,1,Salernitana,1,0,0,0,1
0,2023-05-03 19:00:00,Monza18908,Monza,h,0.412,1.286,1,1,8,14,0,1,1,Roma,1,0,0,0,1


In [12]:
spa_df = matchstat_generator(spa_active, spa_df)
# with pd.ExcelWriter("../EDA_DF/LaLiga_understat.xlsx") as writer: # For VSCode use locally
with pd.ExcelWriter("EDA_DF/LaLiga_understat.xlsx") as writer: # For GitHub Actions use externally
    spa_df.to_excel(writer, sheet_name="gamesheet")
spa_df

,date,ID,Team,H/A,xG,xGC,G,GC,S,SC,QS,QSC,Pts,Opp,BTTS,Blank,CS,first_goal,first_conc
0,2024-04-20 14:15:00,Rayo Vallecano23004,Rayo Vallecano,h,0.560,0.931,2,1,17,6,0,2,3,Osasuna,1,0,0,0,1
0,2024-04-20 14:15:00,Osasuna23004,Osasuna,a,0.931,0.560,1,2,6,17,2,0,0,Rayo Vallecano,1,0,0,1,0
0,2024-04-20 19:00:00,Girona23002,Girona,h,2.309,1.824,4,1,10,12,4,3,3,Cadiz,1,0,0,1,0
0,2024-04-20 19:00:00,Cadiz23002,Cadiz,a,1.824,2.309,1,4,12,10,3,4,0,Girona,1,0,0,0,1
0,2024-04-20 16:30:00,Valencia23001,Valencia,h,0.951,1.821,1,2,8,12,1,3,0,Real Betis,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2023-05-03 20:00:00,Cadiz19283,Cadiz,a,0.026,4.282,1,5,2,18,0,6,0,Atletico Madrid,1,0,0,0,1
0,2023-05-04 20:00:00,Athletic Club19282,Athletic Club,h,1.018,0.756,0,1,20,12,0,0,0,Real Betis,0,1,0,0,1
0,2023-05-04 20:00:00,Real Betis19282,Real Betis,a,0.756,1.018,1,0,12,20,0,0,3,Athletic Club,0,0,1,1,0
0,2023-05-01 19:00:00,Sevilla19281,Sevilla,h,2.352,1.134,0,2,23,9,3,1,0,Girona,0,1,0,0,1
